#### DML: COUNT, DISTINCT, LIMIT

Retrieve the number of distributors distinctly who distributed films acted by Clint Eastwood as 1st actor.</p>
`SELECT COUNT(DISTINCT Distributor) FROM FilmLocations WHERE Actor1="Clint Eastwood";`

跳过第一项</p>
`SELECT * FROM FilmLocations LIMIT 3 OFFSET 1;`

#### DML: INSERT, UPDATE, DELETE

`INSERT INTO Instructor(ins_id, lastname, firstname, city, country)
VALUES (5, 'John', 'Doe', 'Sydney', 'AU'),(6, 'Jane', 'Doe', 'Dhaka', 'BD');
`

`UPDATE Instructor 
SET city='Dubai' ,country='AE'
WHERE ins_id="5";`

`DELETE FROM instructor
WHERE firstname = 'Hima';`

#### DDL: CREATE, INSERT, ALTER, UPDATE, TRUNCATE, DROP

`CREATE TABLE PETSALE (
    ID INTEGER NOT NULL,
    PET CHAR(20),
    SALEPRICE DECIMAL(6,2),
    PROFIT DECIMAL(6,2),
    SALEDATE DATE
    );`
    
`CREATE TABLE PET (
    ID INTEGER NOT NULL,
    ANIMAL VARCHAR(20),
    QUANTITY INTEGER
    );`
    
`INSERT INTO PETSALE VALUES
    (1,'Cat',450.09,100.47,'2018-05-29'),
    (2,'Dog',666.66,150.76,'2018-06-01'),
    (3,'Parrot',50.00,8.9,'2018-06-04'),
    (4,'Hamster',60.60,12,'2018-06-11'),
    (5,'Goldfish',48.48,3.5,'2018-06-14');`
    
`INSERT INTO PET VALUES
    (1,'Cat',3),
    (2,'Dog',4),
    (3,'Hamster',2);`

`ALTER TABLE PETSALE
ADD COLUMN QUANTITY INTEGER;`

`UPDATE PETSALE SET QUANTITY = 9 WHERE ID = 1;
UPDATE PETSALE SET QUANTITY = 3 WHERE ID = 2;
UPDATE PETSALE SET QUANTITY = 2 WHERE ID = 3;
UPDATE PETSALE SET QUANTITY = 6 WHERE ID = 4;
UPDATE PETSALE SET QUANTITY = 24 WHERE ID = 5;`   

`ALTER TABLE PETSALE
DROP COLUMN PROFIT;`

`ALTER TABLE PETSALE
ALTER COLUMN PET SET DATA TYPE VARCHAR(20);`

`ALTER TABLE PETSALE
RENAME COLUMN PET TO ANIMAL;`

`TRUNCATE TABLE PET IMMEDIATE;`

`DROP TABLE PET;`


####  GROUP BY, HAVING, ORDER BY

use foreign key to select</P>
`SELECT D.DEP_NAME , E.F_NAME, E.L_NAME
FROM EMPLOYEES as E, DEPARTMENTS as D
WHERE E.DEP_ID = D.DEPT_ID_DEP
ORDER BY D.DEP_NAME, E.L_NAME DESC;`

`SELECT DEP_ID, COUNT(*) AS "NUM_EMPLOYEES", AVG(SALARY) AS "AVG_SALARY"
FROM EMPLOYEES
GROUP BY DEP_ID
HAVING COUNT(*) < 4
ORDER BY AVG_SALARY;`

#### sub-queries, nested selection and multiple tables (alias)

`select JOB_TITLE, MIN_SALARY,MAX_SALARY,JOB_IDENT 
from jobs 
where JOB_IDENT IN 
(select JOB_ID from employees where SALARY > 70000 );`

implicit join</p>
`select E.EMP_ID,E.F_NAME,E.L_NAME, J.JOB_TITLE 
from employees E, jobs J where E.JOB_ID = J.JOB_IDENT;`

explicit join</p>
`select * from
table a inner join table b on a.id = b.id;`

#### python connecting to db2

`%load_ext sql
%sql ibm_db_sa://xcx21383:obn6EJZ71vblyuAF@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB?security=SSL`

#### show a list of tables in database

Get a list of columns in the database:
Show tables in table schema: </p>
`%sql select * from syscat.tables where tabschema='XCX21383'`

To obtain specific column properties:  </p>
`%sql select distinct(name),coltype, length from sysibm.syscolumns where tbname = 'DOGS'` </p>
or </p>
`%sql select colname, typename, length from syscat.columns where tabname = 'DOGS'`

#### remove%

Remove the '%' sign from the above result set for Average Student Attendance column

`%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from CHICAGOPUBLICSCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only`

Which Schools have Average Student Attendance lower than 70%?

`%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGOPUBLICSCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \  #这里不知道为什么转成小数
     order by Average_Student_Attendance`

#### multiple tables

Get the hardship index for the community area which has College Enrollment of 4368

`%sql select hardship_index \
   from chicago_socioeconomic_data CD, CHICAGOPUBLICSCHOOLS CPS \
   where CD.ca = CPS.community_area_number \
   and college_enrollment = 4368`

Get the hardship index for the community area which has the school with the highest enrollment.

`%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from CHICAGOPUBLICSCHOOLS order by college_enrollment desc limit 1 )`

#### Views

`CREATE VIEW EMPSALARY AS 
SELECT EMP_ID, F_NAME, L_NAME, B_DATE, SEX, SALARY
FROM EMPLOYEES;`

`CREATE OR REPLACE VIEW EMPSALARY  AS 
SELECT EMP_ID, F_NAME, L_NAME, B_DATE, SEX, JOB_TITLE, MIN_SALARY, MAX_SALARY
FROM EMPLOYEES, JOBS
WHERE EMPLOYEES.JOB_ID = JOBS.JOB_IDENT;`

`DROP VIEW EMPSALARY;`

#### Procedure

In [ ]:
--#SET TERMINATOR @
CREATE PROCEDURE UPDATE_SALEPRICE ( 
    IN Animal_ID INTEGER, IN Animal_Health VARCHAR(5) )     -- ( { IN/OUT type } { parameter-name } { data-type }, ... )

LANGUAGE SQL                                                -- Language used in this routine
MODIFIES SQL DATA                                           -- This routine will only write/modify data in the table

BEGIN 

    IF Animal_Health = 'BAD' THEN                           -- Start of conditional statement
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE - (SALEPRICE * 0.25)
        WHERE ID = Animal_ID;
    
    ELSEIF Animal_Health = 'WORSE' THEN
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE - (SALEPRICE * 0.5)
        WHERE ID = Animal_ID;
        
    ELSE
        UPDATE PETSALE
        SET SALEPRICE = SALEPRICE
        WHERE ID = Animal_ID;

    END IF;                                                 -- End of conditional statement
    
END
@                                                           -- Routine termination character

In [ ]:
CALL RETRIEVE_ALL;

CALL UPDATE_SALEPRICE(1, 'BAD');        -- Caller query

CALL RETRIEVE_ALL;

DROP PROCEDURE UPDATE_SALEPRICE;

#### Transaction

In [ ]:
--#SET TERMINATOR @
CREATE PROCEDURE TRANSACTION_JAMES                          -- Name of this stored procedure routine

LANGUAGE SQL                                                -- Language used in this routine 
MODIFIES SQL DATA                                           -- This routine will only write/modify data in the table

BEGIN

        DECLARE SQLCODE INTEGER DEFAULT 0;                  -- Host variable SQLCODE declared and assigned 0
        DECLARE retcode INTEGER DEFAULT 0;                  -- Local variable retcode with declared and assigned 0
        DECLARE CONTINUE HANDLER FOR SQLEXCEPTION           -- Handler tell the routine what to do when an error or warning occurs
        SET retcode = SQLCODE;                              -- Value of SQLCODE assigned to local variable retcode
        
        UPDATE BankAccounts
        SET Balance = Balance-1200
        WHERE AccountName = 'James';
        
        UPDATE BankAccounts
        SET Balance = Balance+1200
        WHERE AccountName = 'Shoe Shop';
        
        UPDATE ShoeShop
        SET Stock = Stock-4
        WHERE Product = 'Trainers';
        
        UPDATE BankAccounts
        SET Balance = Balance-150
        WHERE AccountName = 'James';

        
        IF retcode < 0 THEN                                  --  SQLCODE returns negative value for error, zero for success, positive value for warning
            ROLLBACK WORK;
        
        ELSE
            COMMIT WORK;
        
        END IF;
        
END
@                                                            -- Routine termination character

In [ ]:
CALL TRANSACTION_JAMES;  -- Caller query

SELECT * FROM BankAccounts;

SELECT * FROM ShoeShop;

#### INNER JOIN, LEFT/RIGHT OUTER JOIN, FULL OUTER JOIN

`select E.F_NAME,E.L_NAME, JH.START_DATE, J.JOB_TITLE 
from EMPLOYEES as E 
INNER JOIN JOB_HISTORY as JH on E.EMP_ID=JH.EMPL_ID 
INNER JOIN JOBS as J on E.JOB_ID=J.JOB_IDENT
where E.DEP_ID ='5';`

`select E.F_NAME,E.L_NAME,D.DEPT_ID_DEP, D.DEP_NAME
from EMPLOYEES AS E 
FULL OUTER JOIN DEPARTMENTS AS D ON E.DEP_ID=D.DEPT_ID_DEP AND E.SEX = 'M';`